## Broad category model for assay descriptions
The notebook manages the main pipeline for model training in 100% data

In [7]:
import pandas as pd
from pathlib import Path
import os
import json
import spacy
from sklearn.model_selection import RepeatedKFold
import shutil
import subprocess

### Setup

In [8]:
# Set path to python in the environment to use for model training
env_path = "/hps/software/users/chembl/ines/assays_description/bin/python"

In [9]:
#Settings for display (if needed)
pd.set_option('display.max_colwidth', None)  # Set to None to display the full column width
pd.set_option('display.max_rows', None)      # Set to None to display al

### Model training: Binding assays and functional assays

##### Clean up dataset

In [10]:
dataset = pd.read_csv('data/2_broad_category_training_data.csv')

In [11]:
dataset.head()

,assay_id,assay_type,description,label,bao_preferred_term,bao_id
0,868,B,"Inhibition of [3H]8-hydroxy-2-dipropylamino-1,2,3,4-tetrahydronaphthalene binding to 5-hydroxytryptamine 1A receptor in hippocampus region of rat brain; Residual radioligand binding higher than 50%","Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
1,2027,B,Displacement of [3H]-5-HT from human 5-hydroxytryptamine 1D receptor beta,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
2,2430,B,Inhibition constant for in vitro inhibition of [3H]ketanserin binding to rat frontal cortex membranes 5-hydroxytryptamine 2A receptor,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
3,3306,B,Compound was evaluated for the binding affinity against human cloned 5-hydroxytryptamine 4 receptor in HeLa cells using [3H]-LSD as the radioligand,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
4,3703,B,In vitro binding affinity by radioligand binding assay using cell line expressing human 5-hydroxytryptamine 7 receptor; ND means not determined,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776


In [12]:
dataset.value_counts(subset='label')

label
Nucleic acid binding                              176
Protein activity                                  171
Binding affinity, displacement, competition       134
Radioligand competition, displacement, binding    118
Cell phenotype                                    113
Antimicrobial activity                             88
in vivo method                                     63
Name: count, dtype: int64

In [7]:
# Can only train the large categories
chosen_categories = [
    'Nucleic acid binding'
    , 'Protein activity'
    , 'Binding affinity, displacement, competition'
    , 'Radioligand competition, displacement, binding'
    , 'Cell phenotype'
    , 'Antimicrobial activity'
    , 'in vivo method'
]

In [8]:
processed_df = dataset.loc[dataset['label'].isin(chosen_categories)]

In [9]:
value_dict = {
    'Radioligand competition, displacement, binding': {'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Binding affinity, displacement, competition': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 1.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Protein activity': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 1.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'in vivo method': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 1.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Cell phenotype': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 1.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Nucleic acid binding': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 1.0, 'Antimicrobial activity': 0.0}
    , 'Antimicrobial activity': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 1.0}

}

In [10]:
processed_df['cats'] = (
    processed_df['label']
    .apply(lambda x: value_dict[x])
)

/tmp/ipykernel_199556/479963469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['cats'] = (


In [11]:
processed_df.rename(columns={'description': 'text'}, inplace=True)

/tmp/ipykernel_199556/3262683888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df.rename(columns={'description': 'text'}, inplace=True)


In [12]:
processed_df = processed_df[['text', 'cats']]

#### Save training file

In [13]:
mpath = "Model"
dirpath = Path(mpath)
if dirpath.exists():
    shutil.rmtree(dirpath)
os.makedirs(mpath)

# Write to JSONL files
with open(os.path.join(mpath,'assays_train.jsonl'), 'w') as f:
    f.write(processed_df.to_json(orient='records', lines=True))

#### Do training

In [14]:
pwd

'/homes/ines/repos/assays_description/2_broad_assay_category'

In [15]:
#set up path for training data
train = os.path.join('../', mpath,'assays_train.jsonl')

# Chdir to the pipeline template folder
os.chdir('textcat_broad_categories')

os.mkdir('assets')
os.mkdir('training')

#Copy the current input files to the pipeline path
shutil.copy(train, os.path.join('assets/'))

#Run the pipeline
command = f'{env_path} -m weasel run final'
subprocess.run(command, shell=True, capture_output=False, text=True)

#Move outputs to the folder
opath = os.path.join('../', mpath, 'training')
shutil.copytree('training', opath, dirs_exist_ok=True)

# Remove directories to start clean
shutil.rmtree('assets')
shutil.rmtree('training')
shutil.rmtree('corpus')
os.remove('project.lock')

os.chdir('../')

ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat_multilabel']
ℹ Initial learn rate: 0.0
E    #       LOSS TOK2VEC  LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  ------------  -------------  ----------  ------


  0       0          0.58           0.43       52.96    0.53


  1     200         92.62          60.48       61.01    0.61


  2     400         47.02          29.76       79.00    0.79


  4     600         31.08          21.00       89.29    0.89


...

304    7200          1.34           0.04      100.00    1.00


315    7400          2.11           0.03      100.00    1.00


325    7600          3.97           0.04      100.00    1.00


336    7800          1.72           0.02      100.00    1.00
✔ Saved pipeline to output directory
training/model-last


ℹ Running workflow 'final'

=========================== convert-training-only ===========================
Running command: /hps/software/users/chembl/ines/assays_description/bin/python scripts/convert.py en assets/assays_train.jsonl corpus/train.spacy

================================ train-final ================================
Running command: /hps/software/users/chembl/ines/assays_description/bin/python -m spacy train configs/config.cfg --output training/ --paths.train corpus/train.spacy --nlp.lang en --gpu-id -1
